In [8]:
from modules import Path 

from tika import parser
import string
import re
import io
import os


# In dieser Datei soll eine Neue Datei geschrieben.
# Für jeden Jahresabschluss, den diese Datei einließt, soll eine Excel-Arbeitsmappe erstellt 
# werden, die anhand einer KeyWordListe matches abgleicht und den Satz des Matches extrahiert. 

#=============================================================================================
# 0. PFADE SAMMELN
#=============================================================================================

# Pfadliste
pfade = Path.fetch("test_data")
for pfad in pfade:
    print(pfad)

C:\Users\test\Documents\GitHub\hhu_whk_project_restatements\test_data\Bayer AG2009_SR.pdf
C:\Users\test\Documents\GitHub\hhu_whk_project_restatements\test_data\BMW AG2008_SR.pdf
C:\Users\test\Documents\GitHub\hhu_whk_project_restatements\test_data\Deutsche Post AG2010_SR.pdf
C:\Users\test\Documents\GitHub\hhu_whk_project_restatements\test_data\E.ON AG2011_SR.pdf
C:\Users\test\Documents\GitHub\hhu_whk_project_restatements\test_data\Evonik Industries AG2011_SR.pdf
C:\Users\test\Documents\GitHub\hhu_whk_project_restatements\test_data\Fraport AG FFM Airport2012_SR.pdf
C:\Users\test\Documents\GitHub\hhu_whk_project_restatements\test_data\Henkel AG_Co.KGAA2013_SR.pdf
C:\Users\test\Documents\GitHub\hhu_whk_project_restatements\test_data\Linde AG_SR_2014.pdf
C:\Users\test\Documents\GitHub\hhu_whk_project_restatements\test_data\MAN SE_SR_2015.pdf
C:\Users\test\Documents\GitHub\hhu_whk_project_restatements\test_data\RWE AG_SR_2016.pdf
C:\Users\test\Documents\GitHub\hhu_whk_project_restatements\t

In [9]:
# KWL laden:
KWL_raw = []
with open("KWL.txt", "r", encoding="utf-8") as KWL_file:
    for line in KWL_file:
        KWL_raw.append(line.strip())
len(KWL_raw)

70

In [10]:
# wegen dopplungen in der KWL-raw wird diese überarbeitet. 
KWL_set = set()
KWL = []

for key in KWL_raw:
    KWL_set.add(key)
for key in KWL_set:
    KWL.append(key)
len(KWL)

60

In [11]:
KWL

['redigiert',
 'richtig stellen',
 'Neufassung',
 'Abänderung',
 'ausbessern',
 'umformuliert',
 'überprüfen',
 'upgedated',
 'verbessert',
 'ändern',
 'geändert',
 'Nachprüfung',
 'Bearbeitung',
 'umformulieren',
 'aufdatieren',
 'berichtigen',
 'abgeändert',
 'berichtigt',
 'Ausbesserung',
 'bereinigt',
 'korrigiert',
 'Update',
 'Nachbesserung',
 'Anpassung',
 'Revision',
 'Korrektion',
 'überarbeitet',
 'verbessern',
 'überarbeiten',
 'Neuformulierung',
 'beheben',
 'Umformulierung',
 'aktualisiert',
 'aktualisieren',
 'neu formuliert',
 'ausgebessert',
 'Aktualisierung',
 'angepasst',
 'revidiert',
 'Richtigstellung',
 'Bereinigung',
 'richtig gestellt',
 'Verbesserung',
 'Korrektur',
 'bearbeitet',
 'redigieren',
 'neu formulieren',
 'revidieren',
 'updaten',
 'bearbeiten',
 'korrigieren',
 'Überprüfung',
 'abändern',
 'Überarbeitung',
 'Fehlerverbesserung',
 'Änderung',
 'Berichtigung',
 'überholt',
 'bereinigen',
 'anpassen']

In [12]:
# Okay, da waren 10 Schlüssel doppelt belegt. Daher die riesige Anzahl an Matches im ersten Durchgang. 

In [13]:
# 2 Ansätze: 

# Easy Approach: Das Matching ist simpel, jedes Dokument wird in Sätze zerlegt.

# Hard Approach: Das Matching wird komplex, man extrahiert keine Sätze sondern die Postion des 
# matches +- eine Zeichenanzahl. Text kann dabei in Form von Rohdaten bleiben. Vorteil: 
# Man übersieht nicht aus versehen Matches (z.B.) unter Grafiken (sind keine Sätze).

#=============================================================================================
# 1. EASY APPROACH
#=============================================================================================

# Pfadliste
pfade = Path.fetch("test_data")

for file in pfade:
    print(file)
    
    # Jahresabschluss einlesen
    parsedPDF = parser.from_file(file) 

    # Name des Dokuments speichern
    name = os.path.basename(file)[0:-4] #letzter Block um ".pdf" abzuschneiden
    
    # Einzelne Seiten aufrufen
    pages_raw = parsedPDF['content'].split('\n\n\n\n')
    pages = [page for page in pages_raw if len(page)] # HIER SIND NOCH UMLAUTE, bEIM SPEICHERN NACHHER NICHT?
    
    # Seitenzähler innitiieren
    page_number = 0
    
    for page in pages:
        
        page_number += 1
        
        # Text_prep: Absätze (\n), Tabs (\t) und Semikoli raus; Liste aus Sätzen erstellen // Normalerweise schriebt man noch alles klein, aber wegen KWL nicht möglich. 
        text = page.replace('\n', "").replace('\t', "").replace(";", '').strip().lower()
        satz_liste = text.split('.')

        # Satzweise die gesamte KWL checken lassen. Wenn Match, dann Satz m it Nummer in Match_Liste ablegen
        satz_counter = 0
        for satz in satz_liste:
            satz_counter += 1
            for key in KWL:
                if re.search(key, satz):
                    with open("./arbeitsprobe.csv", "a", encoding="utf-8") as File:
                        File.write(u"{};{};{};{};{}\n".format(name, page_number, satz_counter, key, satz))


C:\Users\test\Documents\GitHub\hhu_whk_project_restatements\test_data\Bayer AG2009_SR.pdf
C:\Users\test\Documents\GitHub\hhu_whk_project_restatements\test_data\BMW AG2008_SR.pdf
C:\Users\test\Documents\GitHub\hhu_whk_project_restatements\test_data\Deutsche Post AG2010_SR.pdf
C:\Users\test\Documents\GitHub\hhu_whk_project_restatements\test_data\E.ON AG2011_SR.pdf
C:\Users\test\Documents\GitHub\hhu_whk_project_restatements\test_data\Evonik Industries AG2011_SR.pdf
C:\Users\test\Documents\GitHub\hhu_whk_project_restatements\test_data\Fraport AG FFM Airport2012_SR.pdf
C:\Users\test\Documents\GitHub\hhu_whk_project_restatements\test_data\Henkel AG_Co.KGAA2013_SR.pdf
C:\Users\test\Documents\GitHub\hhu_whk_project_restatements\test_data\Linde AG_SR_2014.pdf
C:\Users\test\Documents\GitHub\hhu_whk_project_restatements\test_data\MAN SE_SR_2015.pdf
C:\Users\test\Documents\GitHub\hhu_whk_project_restatements\test_data\RWE AG_SR_2016.pdf
C:\Users\test\Documents\GitHub\hhu_whk_project_restatements\t

In [14]:
#Analyse der Arbeitsprobe bis hier her:

import pandas as pd


df = pd.read_csv('arbeitsprobe.csv', ';')
df.head()

,name,seite,satz_nummer,key,satz
0,Bayer AG2009_SR,1,2,aktualisiert,die anliegen unserer stakehol-der bilden die ...
1,Bayer AG2009_SR,1,33,ändern,wir möchten frauen in entwick-lungs- und schw...
2,Bayer AG2009_SR,1,89,ändern,69) online-datenbank globesmart zu umgangsfo...
3,Bayer AG2009_SR,1,100,aktualisiert,"22, 52) „bayrisk“ verfahrensanleitung aktual..."
4,Bayer AG2009_SR,2,2,aktualisiert,die anliegen unserer stakehol-der bilden die ...


In [17]:
df.key.value_counts()

ändern           303
verbessern       232
angepasst        133
verbessert       116
überprüfen        52
aktualisiert      48
überarbeitet      44
bereinigt         36
geändert          31
bearbeitet        26
anpassen           8
korrigiert         6
korrigieren        6
beheben            6
aktualisieren      5
bearbeiten         5
überarbeiten       5
Name: key, dtype: int64

In [ ]:
#=============================================================================================
# 1. HRAD APPROACH
#=============================================================================================